In [1]:
from datetime import datetime, timedelta
import pandas as pd
import sqlite3
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import matplotlib.pyplot as plt
import ta
import yfinance as yf
import seaborn as sns
import numpy as np
from queue import Queue
import holidays
import scipy as sp
from itertools import combinations
from pymongo import MongoClient

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

In [2]:
from helper.client import *
from helper.machinelearner import *
from helper.dbmgr import *
from helper.dataprocessor import *

# Create an instance of CoinbaseClient
client = CoinbaseClient(api_key, api_secret)

# Get and format data from CB

In [3]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["mydatabase"]
mycol = mydb["customers"]
mydict = { "name": "John", "address": "Highway 37" }

x = mycol.insert_one(mydict)

print(x.inserted_id)

65d463aeafab67308d1775ba


In [4]:
unique_pids = client.get_unique_product_ids()
unique_pids = [f"{i}-{'USDC'}" for i in unique_pids]

# start_time = datetime(2022, 10, 1)
start_time = datetime(2024, 2, 14)
end_time = datetime(2024, 2, 15)

db_manager = DatabaseManager(start_time=start_time, end_time=end_time)
print(db_manager.check_row_counts())

db_manager.main(unique_pids)

Checking the number of documents in each collection...
Collection 'crypto_candles_one_minute' has 130761 documents.
Collection 'crypto_candles_five_minute' has 26473 documents.
Collection 'crypto_candles_fifteen_minute' has 9592 documents.
Collection 'crypto_candles_thirty_minute' has 4767 documents.
Collection 'crypto_candles_one_hour' has 2308 documents.
Collection 'crypto_candles_two_hour' has 1193 documents.
Collection 'crypto_candles_six_hour' has 384 documents.
Collection 'crypto_candles_one_day' has 48 documents.
None
Preparing to fetch and store data...


# Data Processing

In [ ]:
db_uri = 'mongodb://localhost:27017/'  
db_name = 'crypto_data'  

fe = FeatureEngineer(db_name)
processed_data = fe.run()